In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, Dates, Plots, TimeSeries, JSON, Distributions, ExpectationMaximization, StatsBase

In [2]:
data_directory = joinpath("/projects", "emco4286", "data", "gads", "trajectories", "ct", "long")
files = glob("gen_*_class_CT_rating_*_state_Texas.csv", data_directory)

168-element Vector{String}:
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_2_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 31 bytes ⋯ "ass_CT_rating_2_state_Texas.csv"
 ⋮
 "/projects/emco4286/data/gads/tr" ⋯ 30 bytes ⋯ "ass_CT_rating_1_state_Texas.csv"
 "/projects/emco4286/data/gads/tr" ⋯ 30 bytes ⋯ "ass_CT_rating_1_st

In [3]:
ph = 8760

foh = 40.753
num_fo = 0.518

poh = 43.111
num_po = 0.401

moh = 8.038
num_mo = 0.146

fo_ttf_rate = (ph - moh - poh)/num_fo
po_ttf_rate = (ph - moh - foh)/num_po
mo_ttf_rate = (ph - foh - poh)/num_mo

@printf "TTF FO: %0.2f hours\n" fo_ttf_rate
@printf "TTF PO: %0.2f hours\n" po_ttf_rate
@printf "TTF MO: %0.2f hours\n" mo_ttf_rate

TTF FO: 16812.45 hours
TTF PO: 21723.71 hours
TTF MO: 59425.59 hours


In [4]:
ttf_values_fo_vec = Any[]
ttr_values_fo_vec = Any[]

ttf_values_po_vec = Any[]
ttr_values_po_vec = Any[]

ttf_values_mo_vec = Any[]
ttr_values_mo_vec = Any[]

outage_types_vec = []

for f in files

    df = DataFrame(CSV.File(f))

    if size(df)[1] < 1
        continue
    end

    outage_start = findall(diff(df[!, :y1]) .== -1)
    outage_end = findall(diff(df[!, :y1]) .== 1)

    # If the plant data starts with an outage, don't count the end of the first outage because we won't know the actual TTR
    if df[1, :y1] == 0
        outage_end = outage_end[begin + 1: end]
    end

    if length(outage_end) < length(outage_start)
        outage_start = outage_start[begin: end-1]
    elseif length(outage_end) > length(outage_start)
        outage_end = outage_end[begin + 1: end]
    end

    ttr_values = outage_end .- outage_start

    if length(filter(x -> x < 0, ttr_values)) > 0
        print(f)
        break
    end
    
    ttf_values_all_types = outage_start[begin + 1: end] - outage_end[begin:end - 1]

    outage_types = df[outage_start .+ 1, :y2]
    push!(outage_types_vec, outage_types)
    
    fo_idx = findall(outage_types .== 2)
    os_fo = outage_start[fo_idx]
    oe_fo = outage_end[fo_idx]

    fo_idx_mod = filter(x -> x > 0, fo_idx .- 1)

    ttf_values_fo = os_fo[begin + 1: end] - oe_fo[begin:end - 1] # ttf_values_all_types[fo_idx_mod]
    ttr_values_fo = ttr_values[fo_idx]

    push!(ttf_values_fo_vec, ttf_values_fo)
    push!(ttr_values_fo_vec, ttr_values_fo)

    po_idx = findall(outage_types .== 3)
    os_po = outage_start[po_idx]
    oe_po = outage_end[po_idx]

    po_idx_mod = filter(x -> x > 0, po_idx .- 1)

    ttf_values_po = os_po[begin + 1: end] - oe_po[begin:end - 1] # ttf_values_all_types[po_idx_mod]
    ttr_values_po = ttr_values[po_idx]

    push!(ttf_values_po_vec, ttf_values_po)
    push!(ttr_values_po_vec, ttr_values_po)

    mo_idx = findall(outage_types .== 4)
    os_mo = outage_start[mo_idx]
    oe_mo = outage_end[mo_idx]

    mo_idx_mod = filter(x -> x > 0, mo_idx .- 1)

    ttf_values_mo = os_mo[begin + 1: end] - oe_mo[begin:end - 1] # ttf_values_all_types[mo_idx_mod]
    # ttf_values_mo = os_mo[begin + 1: end] - oe_mo[begin:end - 1]
    ttr_values_mo = ttr_values[mo_idx]

    push!(ttf_values_mo_vec, ttf_values_mo)
    push!(ttr_values_mo_vec, ttr_values_mo)

end

In [5]:
a = reduce(vcat, outage_types_vec)
total = sum([count(==(element),a) for element in unique(a) ])
my_dict = Dict([element => count(==(element),a) for element in unique(a)])

fo_proportion = (my_dict[2.0])/total
po_proportion = my_dict[3.0]/total
mo_proportion = my_dict[4.0]/total

ttr_values_fo_vec = reduce(vcat, ttr_values_fo_vec)
ttr_values_po_vec = reduce(vcat, ttr_values_po_vec)
ttr_values_mo_vec = reduce(vcat, ttr_values_mo_vec)

ttr_f = fit_mle(Exponential(1/foh), ttr_values_fo_vec)
ttr_p = fit_mle(Exponential(1/poh), ttr_values_po_vec)
ttr_m = fit_mle(Exponential(1/moh), ttr_values_mo_vec)

print("$(round(ttr_f.θ, digits=2)), $(round(ttr_p.θ, digits=2)), $(round(ttr_m.θ, digits=2))")

69.91, 186.22, 53.89

In [6]:
round.([fo_proportion, mo_proportion, po_proportion], digits=3)

3-element Vector{Float64}:
 0.51
 0.243
 0.247

In [7]:
ttf_values_fo_vec = reduce(vcat, ttf_values_fo_vec)
ttf_values_po_vec = reduce(vcat, ttf_values_po_vec)
ttf_values_mo_vec = reduce(vcat, ttf_values_mo_vec)

ttf_f = fit_mle(Exponential(1/fo_ttf_rate), ttf_values_fo_vec)
ttf_p = fit_mle(Exponential(1/po_ttf_rate), ttf_values_po_vec)
ttf_m = fit_mle(Exponential(1/mo_ttf_rate), ttf_values_mo_vec)

print("$(round(ttf_f.θ, digits=2)), $(round(ttf_p.θ, digits=2)), $(round(ttf_m.θ, digits=2))")

1494.58, 3025.05, 2645.6

In [9]:
all_ttf = reduce(vcat, [ttf_values_fo_vec, ttf_values_po_vec, ttf_values_mo_vec])
all_ttr = reduce(vcat, [ttr_values_fo_vec, ttr_values_po_vec, ttr_values_mo_vec]);

In [10]:
mix_ttf_guess = MixtureModel([ttf_f, ttf_m, ttf_p], [fo_proportion, mo_proportion, po_proportion])
mix_ttf_mle, info = fit_mle(mix_ttf_guess, all_ttf, infos = true)

(MixtureModel{Exponential{Float64}}(K = 3)
components[1] (prior = 0.1599): Exponential{Float64}(θ=57.44705773496699)
components[2] (prior = 0.6130): Exponential{Float64}(θ=1554.503072449798)
components[3] (prior = 0.2272): Exponential{Float64}(θ=5200.012325768773)
, Dict{String, Any}("iterations" => 115, "converged" => true, "logtots" => [-102328.625596239, -102027.83408974542, -101863.65296386133, -101775.04716768942, -101713.86075373684, -101662.13490715757, -101614.88553202005, -101570.98560817835, -101530.29335726396, -101492.82585526463  …  -101013.89500214066, -101013.89289090739, -101013.89098879349, -101013.88927079272, -101013.88771490098, -101013.88630175559, -101013.88501431771, -101013.88383759336, -101013.88275838757, -101013.88176508833]))

In [13]:
mybins = 0:10:maximum(all_ttf)
histogram(all_ttf, bins=mybins, linewidth=0.05, normalize=:pdf, xlims=(0, 2000), labels="Original data", xlabel="Time to failure [hours]", ylabel="Probability density function", title="CT plant outages")
plot!(mybins, pdf(mix_ttf_guess, mybins), label="Guess fit", linewidth=3)
plot!(mybins, pdf(mix_ttf_mle, mybins), label="MLE fit", linewidth=3)
savefig("/home/emco4286/gads-analysis/visualization/ttf_ct_mixed_exp_fit_guess_vs_mle.png")

"/home/emco4286/gads-analysis/visualization/ttf_ct_mixed_exp_fit_guess_vs_mle.png"

In [11]:
mix_ttr_guess = MixtureModel([ttr_f, ttr_m, ttr_p], [fo_proportion, mo_proportion, po_proportion])
mix_ttr_mle, info = fit_mle(mix_ttr_guess, all_ttr, infos = true)

(MixtureModel{Exponential{Float64}}(K = 3)
components[1] (prior = 0.3060): Exponential{Float64}(θ=115.07762177647247)
components[2] (prior = 0.6528): Exponential{Float64}(θ=7.729482013768594)
components[3] (prior = 0.0412): Exponential{Float64}(θ=1321.910539373515)
, Dict{String, Any}("iterations" => 44, "converged" => true, "logtots" => [-59698.78700463603, -58510.8634868609, -58013.71688591582, -57755.8464585527, -57645.60840989606, -57572.71616733383, -57497.12553324115, -57408.80759942944, -57307.76640244455, -57199.86876206647  …  -56657.671406385736, -56657.64958367816, -56657.63507409049, -56657.62543028075, -56657.61902235383, -56657.614765538616, -56657.611938256734, -56657.61006073165, -56657.60881407405, -56657.60798639187]))

In [15]:
mybins = 0:5:maximum(all_ttr)
histogram(all_ttr, bins=mybins, linewidth=0.05, normalize=:pdf, labels="Original data", xlims=(0, 500), xlabel="Time to repair [hours]", ylabel="Probability density function", title="CT plant outages")
plot!(mybins, pdf(mix_ttr_guess, mybins), label="Guess fit", linewidth=3)
plot!(mybins, pdf(mix_ttr_mle, mybins), label="MLE fit", linewidth=3)
savefig("/home/emco4286/gads-analysis/visualization/ttr_ct_mixed_exp_fit_guess_vs_mle.png")

"/home/emco4286/gads-analysis/visualization/ttr_ct_mixed_exp_fit_guess_vs_mle.png"

In [16]:
ttr_dict = Dict(fieldnames(MixtureModel) .=> getfield.(Ref(mix_ttr_mle), fieldnames(MixtureModel)))
ttf_dict = Dict(fieldnames(MixtureModel) .=> getfield.(Ref(mix_ttf_mle), fieldnames(MixtureModel)))
ct_dict = Dict("ttf" => ttf_dict, "ttr" => ttr_dict)

open("/home/emco4286/gads-analysis/exp_models/ct.json","w") do f 
    JSON.print(f, ct_dict) 
end
